In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout,Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np
ipl_data = pd.read_csv("C:\\Users\\jimen\\PS_project\\model_flask\\ipl_newdata.csv")
data=pd.DataFrame(ipl_data)
data.columns
ipl_data['over'] = ipl_data['over'] / 20.0  # Assuming max overs = 20
ipl_data['ball'] = ipl_data['ball'] / 6.0   
data.drop("match_id",axis=1,inplace=True)
data['batting_team'] = data['batting_team'].replace('Rising Pune Supergiant', 'Lucknow Supergiant')
data['bowling_team'] = data['bowling_team'].replace('Rising Pune Supergiant', 'Lucknow Supergiant')
data['batting_team'] = data['batting_team'].replace('Gujarat Lions', 'Gujarat Giants')
data['bowling_team'] = data['bowling_team'].replace('Gujarat Lions', 'Gujarat Giants')
data

,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,run_per_ball,total,score,crr
0,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,207,0,0.000000
1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,207,0,0.000000
2,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,4,207,4,1.333333
3,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,207,4,1.000000
4,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,2,207,6,1.200000
...,...,...,...,...,...,...,...,...,...,...,...,...
179073,2,Chennai Super Kings,Mumbai Indians,20,2,RA Jadeja,SR Watson,SL Malinga,1,157,152,7.238095
179074,2,Chennai Super Kings,Mumbai Indians,20,3,SR Watson,RA Jadeja,SL Malinga,2,157,154,7.000000
179075,2,Chennai Super Kings,Mumbai Indians,20,4,SR Watson,RA Jadeja,SL Malinga,1,157,155,6.739130
179076,2,Chennai Super Kings,Mumbai Indians,20,5,SN Thakur,RA Jadeja,SL Malinga,2,157,157,6.541667


In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Assuming 'data' is your DataFrame containing both categorical and numerical columns
categorical_cols = ['batting_team', 'bowling_team', 'batsman', 'non_striker', 'bowler']
numerical_cols = ['inning', 'over', 'ball', 'run_per_ball', 'score', 'crr']

# One-Hot Encoding for categorical columns
categorical_data = data[categorical_cols]
encoder = OneHotEncoder(sparse_output=False)
encoded_categorical_data = encoder.fit_transform(categorical_data)
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Scaling the numerical columns


# Combine the numerical and encoded categorical data
final_data = pd.concat([data[numerical_cols], encoded_categorical_df], axis=1)

# Define target variable 'total' and scale it
y = data['total']


# Prepare sequences for LSTM
sequence_length = 1
X_sequences, y_sequences = [], []

# Loop through the data to create sequences of length 'sequence_length'
for i in range(len(final_data) - sequence_length):
    X_sequences.append(final_data.iloc[i:i + sequence_length].values)  # Extract sequences of features
    y_sequences.append(y[i + sequence_length])  # Corresponding target for the sequence

# Convert to numpy arrays
X = np.array(X_sequences, dtype='float32')
y = np.array(y_sequences, dtype='float32')

# Print shapes to verify
print(f"Shape of X: {X.shape}")
print(f"Shape of y: {y.shape}")


Shape of X: (179077, 1, 1468)
Shape of y: (179077,)


In [4]:
# Split the data
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print shapes to verify
print(f"X_train shape: {X_train.shape}, X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")

X_train shape: (143261, 1, 1468), X_test shape: (35816, 1, 1468)
y_train shape: (143261,), y_test shape: (35816,)


In [5]:
model = Sequential([
    LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae']) 
model.summary()

C:\Users\jimen\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 100)                 │         627,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 627,701 (2.39 MB)

 Trainable params: 627,701 (2.39 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# train
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 54s 11ms/step - loss: 3174.1052 - mae: 36.9845 - val_loss: 524.9729 - val_mae: 16.8740
Epoch 2/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 47s 10ms/step - loss: 484.8242 - mae: 16.2012 - val_loss: 442.4969 - val_mae: 15.4021
Epoch 3/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 48s 11ms/step - loss: 427.2047 - mae: 15.1427 - val_loss: 405.7322 - val_mae: 14.6030
Epoch 4/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 47s 11ms/step - loss: 382.7218 - mae: 14.3452 - val_loss: 370.3318 - val_mae: 13.9305
Epoch 5/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 45s 10ms/step - loss: 336.6491 - mae: 13.4347 - val_loss: 327.0403 - val_mae: 13.1834
Epoch 6/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 44s 10ms/step - loss: 302.7667 - mae: 12.7265 - val_loss: 300.2908 - val_mae: 12.5177
Epoch 7/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 47s 10ms/step - loss: 270.5464 - mae: 12.0003 - val_loss: 269.7698 - val_mae: 11.8855
Epoch 8/20
4477/4477 ━━━━━━━━━━━━━━━━━━━━ 45s 10ms/step - loss: 240.6459 - mae: 11.2910 - val_loss: 247

In [7]:
loss = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")


1120/1120 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 121.8469 - mae: 7.4936
Test Loss: [120.52494812011719, 7.511495590209961]


In [8]:
predictions = model.predict(X_test)


1120/1120 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step


In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Predictions from the model
predictions = model.predict(X_test)
# Calculate metrics
mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test,predictions)

# Print results
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R² Score: {r2}")


1120/1120 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step
Mean Absolute Error (MAE): 7.511496067047119
Mean Squared Error (MSE): 120.52497863769531
Root Mean Squared Error (RMSE): 10.978386878967285
R² Score: 0.8696328997612


In [11]:
import pickle
# Save the One-Hot Encoder
with open('one_hot_encoder.pkl', 'wb') as encoder_file:
    pickle.dump(encoder, encoder_file)

# Save the Model
with open('model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler

# Load the pre-trained model, encoder, and scaler
with open('model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

with open('one_hot_encoder.pkl', 'rb') as encoder_file:
    encoder = pickle.load(encoder_file)



# Example user input
user_input = [
    ['Sunrisers Hyderabad', 'Royal Challengers Bangalore', 'DA Warner', 'S Dhawan', 'TS Mills', 1, 15, 3, 6.5, 60, 6.0]
]

# Convert the user input to a DataFrame (to make it consistent with the data you used for training)
user_input_df = pd.DataFrame(user_input, columns=['batting_team', 'bowling_team', 'batsman', 'non_striker', 'bowler', 'inning', 'over', 'ball', 'run_per_ball', 'score', 'crr'])
print(user_input_df)
# Separate the categorical and numerical columns
categorical_cols = ['batting_team', 'bowling_team', 'batsman', 'non_striker', 'bowler']
numerical_cols = ['inning', 'over', 'ball', 'run_per_ball', 'score', 'crr']

# Apply OneHotEncoder to the categorical columns (using the fitted encoder)
encoded_categorical_data = encoder.transform(user_input_df[categorical_cols])
encoded_categorical_df = pd.DataFrame(encoded_categorical_data, columns=encoder.get_feature_names_out(categorical_cols))

# Combine the scaled numerical data and encoded categorical data
final_input = pd.concat([user_input_df[numerical_cols], encoded_categorical_df], axis=1)
# Convert the final input to numpy array for the model
final_input_np = np.array(final_input)
final_input_np = final_input_np.reshape((1, 1, 1468))

# Make the prediction
prediction = model.predict(final_input_np)

# Print the prediction
print(f"Predicted Output: {prediction}")


          batting_team                 bowling_team    batsman non_striker  \
0  Sunrisers Hyderabad  Royal Challengers Bangalore  DA Warner    S Dhawan   

     bowler  inning  over  ball  run_per_ball  score  crr  
0  TS Mills       1    15     3           6.5     60  6.0  
   inning  over  ball  run_per_ball  score  crr  \
0       1    15     3           6.5     60  6.0   

   batting_team_Chennai Super Kings  batting_team_Deccan Chargers  \
0                               0.0                           0.0   

   batting_team_Delhi Capitals  batting_team_Delhi Daredevils  ...  \
0                          0.0                            0.0  ...   

   bowler_WPUJC Vaas  bowler_Washington Sundar  bowler_Y Gnaneswara Rao  \
0                0.0                       0.0                      0.0   

   bowler_Y Nagar  bowler_Y Venugopal Rao  bowler_YA Abdulla  \
0             0.0                     0.0                0.0   

   bowler_YK Pathan  bowler_YS Chahal  bowler_Yuvraj Singh  